In [1]:
# Importando librerias 
%reset -f

# Rutinas que requiere el sistema. 
import numpy as np
import matplotlib.pyplot as plt

# Esto controla el tamaño de las figuras en el script 
plt.rcParams['figure.figsize'] = (16,8)

import ipywidgets as ipw
from ipywidgets import widgets, interact_manual
from IPython.display import Image

# Las rutinas que calculan posiciones y velocidades
from F_FreeFall import FreeFall
from FF_ideal import FF_V

# Esto es para poder correr todo en linea
ipw.interact_manual.opts['manual_name'] = "CALCULAR!"
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})


# **CAÍDA LIBRE DE UNA ESFERA EN UN FLUIDO**

<div  style="text-align: justify" >El estudio del movimiento de partículas esféricas dentro de un fluido es de gran utilidad para revisar los conceptos de esfuerzo cortante, número de Reynolds y efectos de resistencia al movimiento por parte de un fluido a un cuerpo sólido. Para el experimento que se propone, imagine que una esfera de diámetro $D_p$ y densidad $\rho_P$ que parte desde una posición $z \ = \ 0 \ m$ ubicada en la parte alta de un espacio infinto lleno de un fluido con densidad $\rho_F$ y viscosidad cinemática $\nu_f$. En el instante en que la partícula es liberada, su velcidad inicial es $v_0 \ = \ 0 \ m/s$ y para efectos prácticos, el eje $z$ del movimiento aumentará hacia abajo en el sentido del movimiento de la esfera (Figura 1).</div>

<img src="201102_FreeFall_Image.png" alt="fishy" class="bg-primary mb-1"  height="100px">

**Figura 1:** Esquema de una esfera cayendo en un fluido (Fuente: Biringen y Chow, 2011).

<div  style="text-align: justify" >Como es de esperarse, la esfera aumentará su velocidad a medida que cae gracias al efecto de la gravedad. No obstante, ese movimiento no será uniformemente acelerado, ya que existen fuerzas viscosas ocasionadas por la interacción entre la superficie de la esfera y el fluido que irán en contra del movimiento e impedirán su aceleración. Dentro de esas fuerzas se encuentran: *i)* La fricción, ocasionada por la viscosidad del fluido en el que está cayendo la esfera, *ii)* la fuerza de flotación, ocasionada por la diferencia de densidades, *iii)* la fuerza de cuerpo acelerando, ocasionada por los flujos que se crean alrededor de la esfera cuando empieza a moverse y *iv)* el arrastre de onda, presente cuando las velocidades del fluido empiezan a aumentar.</div>

**Insertar imagen de ecuaciones de caída libre en el vacío.**

<div  style="text-align: justify" >Por lo tanto, los factores mencionados anteriormente tienen un impacto en las ecuaciones de la caída libre, ya que reducen la velocidad crítica a la cual la esfera puede llegar, y de la misma manera, reduce la cantidad de distancia que estás mismas pueden alcanzar en un mismo intervalo de tiempo mientras caen en el vacío. Cada uno de estos factores tiene un impacto diferente en la posición y velocidad de la particula, hay algunos que se aplican de la misma manera a las diferentes esferas (no son directamente proporcionales a los diferenciales de la masa), pero, a pesar de esto, el factor que pondera más sobre el las variables posición y velocidad, sigue siendo la gravedad. Pues si bien, los números de Reynolds y el arraste de que esta turbulencia genera aumenta conforme se aumenta el radio y la masa, el peso propio de la esfera seguira siendo el mayor factor. Como se muestra en la figura 2 y 3 la modelación teórica de las curvas de masa y velocidad se acercan a un comportamiento de caída libre en vacío mientras la masa aumenta, puesto que la diferencia entra la fuerza gravitacional y la totalidad de las cuatro fuerzas mencionadas (fricción, flotación, fuerza de campo acelerado y arrastre de onda) es directamente proporcional al aumento de la masa de la esfera.</div>


**Variables utilizadas**

A contiunuación se presenta una lista de las variables que se utilizan en el desdarrollo del presente taller con sus respectivas unidades de medida:

$\rho_f \ (kg/m^3) \ = \ $ Densidad del fluido en el que cae la esfera 

$\rho_p \ (kg/m^3) \ = \ $ Densidad de la partícula que cae en el fluido

$D_p \ (mm) \ = \ $ Diámetro de la partícula que cae en el fluido

$\nu_f \ (m^2/s) \ = \ $ Viscosidad cinemática del fluido 

**Suposiciones**

* La posición inicial de la partícula es $z = 0 \ m$ y $z$ aumenta en el sentido del movimiento
* La partícula inicia su movimiento desde el reposo. Es decir, su velocidad inicial es $v_0 = 0 m/s$
* La densidad de la partícula siempre debe ser mayor que la densidad del fluido para garantizar su caída
* La fuerza de fricción ocasionada por el movimiento es proporcional al cuadrado de la velocidad $(F_f \ \alpha \  \lvert v \rvert ^2)$

**Valores iniciales**

Para el desarrollo del presente problema se asume que una esfera de vidrio de diámtro $D_p \ = \ 10 \ mm$ y densidad $\rho_p \ = \ 2200 \ kg/m^3$ cae en agua con densidad $\rho_f \ = \ 1000 \ kg/m^3$ y viscosidad cinemática $\nu_f \ = \ 1,14 \times 10^{-6} \ m^2/s$.

Estos valores se dberán cambiar para observar los cambios en la caída libre de diferentes objetos en diferentes fluidos. El estudiante deberá investigar las diferentes propiedades de los fluidos en cualquier libro de texto de la asignatura. 

In [2]:
# ===========================================================================
# Se define una función que engloba todo y corre el problema de caída libre 
# cuando se definen los valores de los parámetros que se van a variar para
# el desarrollo dle problema
# ===========================================================================
def RUN_ALL(rho_f, rho_p, nu, D):
    
    # Las condiciones iniciales del problema están "hard coded". No se puede
    # cambiar esta situación a menos que se cambien las funciones que hacen 
    # los cálculos (Acá se ponen estas variables que son las mismas de la 
    # rutina de cálculos)
    Z0 = 0
    V0 = 0
    h = 1e-3
    tf = 3
    
    # Se construye la variable ANSW que almacenará los vectores que serán 
    # graficados. EL orden es el siguiente: T, Zi, Vi, Z, V
    ANSW = np.zeros((5, int(tf / h) + 1))
    
    # Esto corre el caso ideal que no tiene fricción (la idea es poder 
    # comparar lo que sucede en los dos casos)
    ANSW[1, :], ANSW[2, :] =  FF_V(Z0, V0, h, tf)
    
    # Esto corre lo referente al caso con fricción y rozamiento
    ANSW[0, :], ANSW[3, :], ANSW[4, :] = FreeFall(rho_f, rho_p, nu, D)
    
    # Llamando a una función que haga gráficas bonitas en cuanto se refiere
    # a colores, fuentes y manejo del espacio
    plotresults(ANSW)
    

In [3]:
# ===========================================================================
# Haciendo la figura yu poniéndola bonita para efectos de poder entrar a 
# hacerla en un app. Vamos a ver si la hacemos funcionar. 
# ===========================================================================
def plotresults(ANSW):
    
    # plt.style.use('ggplot')
    fig = plt.figure(facecolor="white");
    # fig = plt.subplots(nrows = 1, ncols = 2, sharey = True)
        
    # Para reducir espacio
    Labels = ["Posición teórica", "Posición real", "Velocidad Teórica", 
             "Velocidad Real"]
    
    # Librería de colores (a lo mejor y no uso ninguno)
    Colors = ["darkmagenta","darkgreen","seagreen","dodgerblue","dimgrey"]
    FaceColors = ["lavenderblush","honeydew","mintcream","aliceblue","whitesmoke"]
    
    # Haciendo las gráficas por separado para que sea fácil poder cambiar
    # No necesito hacer ciclos porque son muy pocas. 
    # Gráfica de posiciones contra tiempo
    ax1 = plt.subplot(1, 2, 1)
    ax1.plot(ANSW[0, :], ANSW[1, :], label = Labels[0], c = Colors[0], lw = 3, linestyle=':')
    ax1.plot(ANSW[0, :], ANSW[3, :], label = Labels[1], c = Colors[0], lw = 3)
    ax1.set_ylim([0, 4])
    ax1.set_xlim([0, 2.5])
    ax1.set_facecolor(FaceColors[0])
    ax1.set_title('Posición vs tiempo', fontsize = 20)
    ax1.set_xlabel(r'Tiempo $t(s)$', fontsize = 14)
    ax1.set_ylabel(r'Posición $z(m)$', fontsize = 14)
    ax1.grid(True)
        
    # Gráfica de velocidades
    ax2 = plt.subplot(1, 2, 2)
    ax2.plot(ANSW[0, :], ANSW[2, :], label = Labels[2] ,c = Colors[3], lw = 3, linestyle = ':')
    ax2.plot(ANSW[0, :], ANSW[4, :], label = Labels[3], c = Colors[3], lw = 3)
    ax2.set_ylim([0, 4])
    ax2.set_xlim([0, 2.5])
    ax2.set_facecolor(FaceColors[3])
    ax2.set_title('Velocidad vs tiempo', fontsize = 20)
    ax2.set_xlabel(r'Tiempo $t(s)$', fontsize = 14)
    ax2.set_ylabel(r'Velocidad $v(m/s)$', fontsize = 14)
    ax2.grid(True)
        
    # Mostrando los resultados en el notebook
    plt.show()

In [4]:
# ===========================================================================
# Funcion que corre todo lo que he progrmado y hace que salgan los sliders 
# para determinar los diferentes parámetros que gobiernan el fenómeno de la 
# caída libre de cuerpos en un fluido (desde el reposo)
# ===========================================================================

# Descripción de los sliders 
DESCR = [r"$\rho_f \ (kg/m^3)$",\
         r"$\rho_p \ (kg/m^3)$",\
         r"$\nu_f \ (m^2/s)$",\
         r"$D_p \ (mm)$"]

# Correr todo y poner los sliders en la pantalla
interact_manual(RUN_ALL, \
rho_f=widgets.FloatText(description=DESCR[0], min=600, max=2000, value=1000 , readout_format='E'),\
rho_p=widgets.FloatText(description=DESCR[1], min=2000, max=1e4, value=2200 , readout_format='E'),\
nu=widgets.FloatText(description=DESCR[2], min=1e-8, max=0.1, value=1.14e-6 , readout_format='E'),\
D=widgets.FloatText(description=DESCR[3], min=0.1, max=100 , value=10, readout_format='E'));

interactive(children=(FloatText(value=1000.0, description='$\\rho_f \\ (kg/m^3)$'), FloatText(value=2200.0, de…

Para poder ejecutar de forma correcta la práctica usted deberá correr este notebook en línea o tener unas especificaciones similares a las descritas a continuación: 

In [5]:
# Imprimiendo las dependencias para que esto pueda funcionar. 
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p numpy,matplotlib,watermark 

# date
print (" ")
%watermark -u -n -t -z

ModuleNotFoundError: No module named 'watermark'